<a href="https://colab.research.google.com/github/vitorsr/ccd/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Dados de Queimadas na Amazônia

In [1]:
!wget -q -O ccd_2019.zip https://www.dropbox.com/s/7rriacb7c6vzf3m/ccd_2019.zip

!unzip ccd_2019.zip

Archive:  ccd_2019.zip
replace bdmep_meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Bibliotecas Utilizadas

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

O objetivo aqui é analisar a Influência das Queimadas ocorrentes na Floresta Amazônica com variações na temperatura local. Para isto, utilizaremos dois Datasets:


*   O primeiro com Dados de Medições de Temperatura em várias estações do País;
*   O segundo com o resgistro de Queimadas no Brasil.



## Leitura dos Dados

<br><b>variável | descrição | unidade </b></br>
<br>date | data e hora da coleta             -
<br> id | ID da estação de coleta             -
<br>prec | precipitação |       mm
<br>tair | temperatura do ar  | graus Celsius
<br>tw | temperatura de bulbo úmido |  graus Celsius
<br>tmax | temperatura máxima do ar  | graus Celsius
<br>tmin | temperatura mínima do ar  | graus Celsius
<br>urmax | umidade relativa máxima  |             %
<br>patm | pressão atmosférica         |    hPa
<br>pnmm | pressão atmosférica média ao nível do mar  |           hPa
<br>wd | direção do vento        |   graus
<br>wsmax  |  rajadas de vento    |         m/s
<br>n |  horas de sol          |     h
<br>cc |   cobertura de nuvens    |           -
<br>evap  |   evaporação        |      mm
<br>ur  |   umidade relativa          |     %
<br>ws   |   velocidade do vento       |      m/s

In [3]:
df = pd.read_csv("inmetr.csv")
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
0,1970-05-04 00:00:00,83010,NaN,NaN,NaN,32.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.75,NaN
1,1970-05-04 12:00:00,83010,NaN,25.3,24.0,NaN,23.7,90.0,1005.9,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
2,1970-05-04 18:00:00,83010,NaN,29.2,27.1,NaN,NaN,85.0,1004.2,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
3,1970-05-05 00:00:00,83010,NaN,25.0,27.0,32.6,NaN,92.0,1007.5,NaN,NaN,NaN,NaN,8.0,2.4,88.25,NaN
4,1970-05-05 12:00:00,83010,0.0,25.9,24.8,NaN,23.0,91.0,1006.4,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN


In [4]:
df_m = pd.read_csv("bdmep_meta.csv")
df_m.head()

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1
0,83010,-68.733333,-11.016667,260.00,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
1,82704,-72.666667,-7.633333,170.00,Cruzeiro do Sul,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
2,82915,-67.800000,-9.966667,160.00,Rio Branco,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
3,82807,-70.766667,-8.166667,190.00,Tarauacá,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
4,83098,-36.166667,-10.150000,56.13,Coruripe,Alagoas,AL,America/Maceio,-3,America/Maceio,-3


O primeiro arquivo contém os dados principais de Medições Metereológicas, enquanto o segundo apenas serve de referência para a Localização das Estações de Medição.

Os dados apresentados abragem o período entre os anos de 1970 e 2018.

Filtrando apenas para o Mês de Agosto:

In [0]:
mask = (df['date'].str.split('-',n=2,expand = True)[1]=='08')
df = df.loc[mask]

Extraindo o Ano da Coluna Date:

In [6]:
df['date'] = df['date'].str.split('-',n=1,expand = True)[1]
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
267,08-01 00:00:00,83010,NaN,23.5,24.0,33.2,NaN,NaN,1006.6,NaN,NaN,NaN,NaN,8.0,2.2,84.75,NaN
268,08-01 12:00:00,83010,0.0,20.2,20.0,NaN,15.8,98.0,1010.4,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN
269,08-01 18:00:00,83010,NaN,31.5,24.9,NaN,NaN,59.0,1009.2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
270,08-02 00:00:00,83010,NaN,23.3,24.2,33.1,NaN,91.0,1007.2,NaN,NaN,NaN,NaN,8.0,2.7,83.50,NaN
271,08-02 12:00:00,83010,0.0,20.0,19.9,NaN,15.0,99.0,1009.2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN


Extraindo a Hora de Medição da Coluna Date:

In [7]:
df['date'] = df['date'].str.split(' ',n=1,expand = True)[0]
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
267,08-01,83010,NaN,23.5,24.0,33.2,NaN,NaN,1006.6,NaN,NaN,NaN,NaN,8.0,2.2,84.75,NaN
268,08-01,83010,0.0,20.2,20.0,NaN,15.8,98.0,1010.4,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN
269,08-01,83010,NaN,31.5,24.9,NaN,NaN,59.0,1009.2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
270,08-02,83010,NaN,23.3,24.2,33.1,NaN,91.0,1007.2,NaN,NaN,NaN,NaN,8.0,2.7,83.50,NaN
271,08-02,83010,0.0,20.0,19.9,NaN,15.0,99.0,1009.2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN


In [9]:
df = df.groupby(by=['date','id']).mean()
df.head(1000)

prec       tair         tw  ...      evap         ur        ws
date  id                                     ...                               
08-01 82024  6.219512  27.235246  24.882883  ...  2.597143  84.051471  1.025690
      82042  4.133333  26.858163  24.436145  ...  2.345455  83.840000  0.467799
      82093  5.218182  25.945946  24.086486  ...  2.023077  88.363636  0.641026
      82095  5.961905  26.821538  24.107692  ...  2.780000  82.944444  1.318182
      82098  3.716327  28.204027  25.416779  ...  2.765306  81.761364  1.290667
...               ...        ...        ...  ...       ...        ...       ...
08-03 83536  0.000000  22.933333  17.099333  ...  5.573529  58.590000  1.500000
      83538  0.226667  17.738710  13.782203  ...  4.297674  68.807692  1.912523
      83539  0.000000  21.056250  15.862500  ...  5.185714  64.666667  2.500000
      83543  0.282051  23.315044  19.342453  ...  3.438889  70.175676  1.313808
      83546  0.600000        NaN        NaN  ...  3.850000        NaN       NaN

[1000 rows x 15 columns]